In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('opinion_lexicon') #Download Opinion Dictionary
from nltk.corpus import opinion_lexicon
positive_wds = set(opinion_lexicon.positive())
negative_wds = set(opinion_lexicon.negative())

In [ ]:
#Compiling different fragments of tweets

data1=pd.read_csv("/2018/jan_1-14_2018.csv")
data2=pd.read_csv("/2018/jan_15_2018.csv")
data3=pd.read_csv("/2018/feb_march_2018.csv")
data4=pd.read_csv("/2018/tweets_april_may_2018.csv")
data5=pd.read_csv("/2018/june_2018.csv")
data6=pd.read_csv("/2018/july_2018.csv")
data7=pd.read_csv("/2018/aug_sep_2018.csv")
data8=pd.read_csv("/2018/oct_2018.csv")
data9=pd.read_csv("/2018/nov_2018.csv")
data10=pd.read_csv("/2018/dec_2018.csv")

data = pd.concat([data1,data2,data3,data4,data5,data6,data7,data8,data9,data10], axis=0)
data.drop_duplicates()

In [ ]:
data.sort_values(by="Date")

In [ ]:
#Functions to calculate the NLTK sentiments 
def score_sent(sent):
    """Returns a score btw -1 and 1"""
    sent = [e.lower() for e in sent if e.isalnum()]
    total = len(sent)
    pos = len([e for e in sent if e in positive_wds])
    neg = len([e for e in sent if e in negative_wds])
    if total > 0:
        return (pos - neg) / total
    else:
        return 0

def score_review(s):
    sentiment_scores = []
    sents = sent_tokenize(str(s))
    for sent in sents:
        wds = word_tokenize(sent)
        sent_scores = score_sent(wds)
        sentiment_scores.append(sent_scores)
    return sum(sentiment_scores) / len(sentiment_scores)

def score_to_rating(value):
    if value > 0.2:
        return 2
    if value <= 0.2 and value >= -0.2:
        return 1
    else:
        return 0

#Calculating the NLTK sentiments for each tweet
data['Sentiment'] = data['Tweet'].apply(score_review)

In [ ]:
dates=[]
for l in data["Date"]:
    dates.append(datetime.strftime(parse(l),'%Y/%m/%d'))

#Changing the dates to only have the date; Eliminating the time section     
data["Date"]=dates

#Grouping all tweets for a day
datewise_grps = data.groupby(data["Date"])

#Number of tweets for the day
num_tweets = datewise_grps.size()

#Average value for the day
downsampled_data=datewise_grps.mean()

#Adding the count attribute
downsampled_data["Tweets_count"]=num_tweets

#saving the files
downsampled_data.to_csv("filename.csv")

#The same process was applied to the news headlines